In [1]:
from os.path import join, exists, dirname
from glob import glob
import pickle
import mplh.cluster_help as ch
import os
import vireoSNP
import numpy as np
from scipy import sparse
from scipy.io import mmread
import matplotlib.pyplot as plt
from scipy.stats import hypergeom
print(vireoSNP.__version__)

import pandas as pd
import seaborn as sns
from vireoSNP import Vireo
np.set_printoptions(formatter={'float': lambda x: format(x, '.5f')})

from src.external.pyvenn import venn


%load_ext autoreload
%autoreload 2
import src.utils.variant_utils as vu
from src.utils.data_io import wrap_load_mtx_df
from src.utils.variant_utils import get_high, get_low
from icecream import ic
import src.utils.variant_utils as vu
from src.utils.data_io import wrap_load_mtx_df

0.5.6


In [2]:
INDIR = "/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex"
cells_meta_f = "/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simple/vireo/nclones20/cells_meta.tsv"
OUTDIR = "/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simple/vireo/nclones20/clones_qc/"
col = "lineage"
filt_col = "donor"
filt_val = 0


# Params
vars_to_plot=10
sample_names="Control,Flt3l"
N_DONORS=4

In [3]:
cells_meta = pd.read_csv(cells_meta_f, sep='\t')
cells_meta = cells_meta.astype({"donor_index": "Int64", "donor": "Int64", "lineage":"Int64", "lineage_index":"Int64"})
cells_meta = cells_meta.set_index("ID")
cells_meta
#cells_meta.head()

,Unnamed: 0,level_0,index,raw ID,new index,condition,donor,donor_index,lineage,lineage_index
ID,,,,,,,,,,
AAACGAAAGTCGTGAG-1_Control,0,2,2,AAACGAAAGTCGTGAG-1,3,Control,0,1,<NA>,<NA>
AAACGAACACAATAAG-1_Control,1,4,4,AAACGAACACAATAAG-1,5,Control,0,2,<NA>,<NA>
AAACGAAGTAAGTCTC-1_Control,2,9,9,AAACGAAGTAAGTCTC-1,10,Control,0,3,<NA>,<NA>
AAACGAAGTGGATTCT-1_Control,3,12,12,AAACGAAGTGGATTCT-1,13,Control,0,4,<NA>,<NA>
AAACGAATCGCAACTA-1_Control,4,14,14,AAACGAATCGCAACTA-1,15,Control,0,5,3,1
...,...,...,...,...,...,...,...,...,...,...
TTTGGTTGTTAACCAC-1_Flt3l,3746,17371,17371,TTTGGTTGTTAACCAC-1,17372,Flt3l,3,3747,18,174
TTTGTGTAGAGAGTAG-1_Flt3l,3747,17378,17378,TTTGTGTAGAGAGTAG-1,17379,Flt3l,3,3748,<NA>,<NA>
TTTGTGTGTATTCTGG-1_Flt3l,3748,17394,17394,TTTGTGTGTATTCTGG-1,17395,Flt3l,3,3749,<NA>,<NA>


In [6]:
vcf_f = join(INDIR, f"{filt_col}{filt_val}.vcf")
vcf= pd.read_csv(vcf_f,
                sep='\t').fillna("N")
vcf.index = vcf.apply(lambda x: str(x["POS"])+x["REF"] + ">" + x["ALT"],axis=1)
vcf.head()

,#CHROM,POS,REF,ALT,strand_correlation_x,vmr_x,n_cells_over_5_x,n_cells_over_20_x,index,old /data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/Control/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/Control/vireoIn,strand_correlation_y,vmr_y,n_cells_over_5_y,n_cells_over_20_y,old /data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/Flt3l/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/Flt3l/vireoIn,new ID
4949A>G,chrM,49,49A,G,0.905935,0.042452,18.0,0.0,49A>G,1.0,0.83362,0.04046,51.0,0.0,1.0,1
6464C>T,chrM,64,64C,T,0.986767,0.685692,1639.0,1527.0,64C>T,2.0,0.982849,0.57238,4015.0,3670.0,2.0,2
7373A>C,chrM,73,73A,C,1.0,0.0679,3.0,0.0,73A>C,3.0,1.0,0.030665,3.0,0.0,3.0,3
7373A>G,chrM,73,73A,G,0.988898,0.233772,4897.0,4799.0,73A>G,4.0,0.985724,0.157778,9817.0,9564.0,4.0,4
7878C>T,chrM,78,78C,T,0.974539,0.039218,7.0,0.0,78C>T,5.0,N,N,N,N,N,5


In [12]:
af, dp = wrap_load_mtx_df(INDIR, oth_f=False, prefix=f"{filt_col}{filt_val}",
                     columns=('Variant', 'Cell', 'integer'), inc_af=True,
                     as_dense=True, var_names=True, vcf_prefix=f"{filt_col}{filt_val}",
                     verbose=True, cell_names=True)

ic| prefix: 'donor0'


FileNotFoundError: [Errno 2] No such file or directory: '/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/cellSNP.samples.tsv'

In [7]:
variant_df = vu.type_of_variants(AF_df.index)

variants_box= vu.variants_dense(AF_df, vars_to_plot, samples_d=cells_meta['condition'], donors_d=cells_meta['donor'],
                              variant_d=variant_df['variant type'])
variants_box

NameError: name 'AF_df' is not defined